In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from scipy.special import erf

pi = np.pi
exp = np.exp
sqrt = np.sqrt
hbar = 1.0546e-34  # reduced Planck constant
options = Options()

In [2]:
Nq = 2  # number of levels in qubit

wq = 7e9*2*pi  # qubit frequency
wp = wq        # pump frequency

kq = 50e3*2*pi  # photon emission frequency of qubit (qubit decay rate)

Ec = 200e6*2*pi  # Anharmonicity

t0 = 0            # start Gaussian pulse Omega
t1 = t0 + 50e-9   # end Gaussian pulse Omega
t2 = t1 + 10e-9   # start square pulse at resonator 1
t3 = t2 + 300e-9  # end square pulse at resonator 1
t4 = t3 + 10e-9   # start square pulse at resonator 2
t5 = t4 + 300e-9  # end square pulse at resonator 2
t6 = t5 + 10e-9   # t0
tg = 10e-9        # Gaussian rise and fall time of square pulse

N_repeat = 1  # number of times to repeat cycle
Q = 3  # number of std's in Gaussian curves, rises and falls

In [3]:
# Convention:
# [qubit, resonator 1, resonator 2]

# Qubit operators
b = destroy(Nq)
nq = b.dag()*b
sx = sigmax() # x-projection of qubit level
sy = sigmay() # y-projection of qubit level
sz = sigmaz() # z-projection of qubit level

# Jaynes-Cummings Hamiltonian
Hjc = - Ec/2*b.dag()*b.dag()*b*b
"""
Does the derivative term of the unity operator still
have to be added?
"""

# Hamiltonian arguments
H_args = {"t0" : t0, "t1" : t1, "t2" : t2, "t3" : t3, 
          "t4" : t4, "t5" : t5, "t6" : t6, "tg" : tg,
          "Q"  : Q}

In [12]:
def pump_strength(args):
    t0 = args['t0']
    t1 = args['t1']
    Q  = args['Q']
    tau = t1-t0      # pulse duration
    std = tau/(2*Q)  # standard deviation
    time = lambda t : t/(std*sqrt(2))  # t' to pass to error function
    
    integral = sqrt(2*pi)*std*erf(time(tau)/2)
    Omega = pi/integral
    return Omega

# Pump Hamiltonian
Omega = pump_strength(H_args)
H_args['Omega'] = Omega
Hp = (wp-Omega)/2*(b + b.dag()) #- wp*b.dag()*b  # including rotating frame approximation
print(wp)
print(Omega)
print(wp-Omega)

43982297150.2571
150804838.788
43831492311.5


In [9]:
# Collapse operators, losses
cq = sqrt(kq)*b   # qubit
c_ops = [cq]

# Expectation operators
e_ops = [nq, sx, sy, sz]

In [10]:
def pump(t, args):
    t0 = args['t0']  # start of pulse
    t1 = args['t1']  # end of pulse
    Q  = args['Q']
    Omega = args['Omega']  # coupling between qubit and pump field
    """
    Do not divide through 2pi in order to flip qubit to other state.
    """
    
    mu = (t1-t0)/2  # pulse center in time domain
    std = (t1-t0)/(2*Q)  # standard deviation
    confine = np.heaviside((t-t0), 0) - np.heaviside((t-t1), 0)  # entire pulse
    
    pulse = Omega*exp(-(t-mu)**2/(2*std**2))*confine
    return pulse

In [11]:
H = [Hjc, [Hp, pump]]  # complete Hamiltonian
tlist = np.linspace(0, N_repeat*t6, 1000)  # discrete time steps for which to store the output
psi0 = basis(Nq,0)  # initial state
options.nsteps = 1e5  # max number of steps to take for each interval
options.store_states=True

result = mesolve(H, psi0, tlist, c_ops, e_ops=e_ops, args=H_args, options=options)

Exception: ODE integration error: Try to increase the allowed number of substeps by increasing the nsteps parameter in the Options class.

In [ ]:
pulse = pump(tlist, H_args)
plt.plot(tlist, pulse)

In [ ]:
plt.plot(result.expect[3])

In [ ]:
"""
If the qubit is in its excited state it will loose energy due to the collapse operator,
but no cavities are simulated. Where does the energy flow to?
Apparently, QuTiP does not consider energy conservation?
"""

In [ ]:
erf(3)